In [1]:
!pip install torch-geometric
!pip install utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.1/231.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.1/344.1 kB 21.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13906 sha256=c7acdc44ee1c8172fc6c8b2ce57d6642d187812a9ef95e5c764d57516863aa56
  Stored in directory: /root/.cache/pip/wheels/15/0c/b3/674aea8c5d91c642c817d4d630bd58faa316724b136844094d
Successfully built utils


In [2]:

from pathlib import Path
import argparse
import time

import torch
import random

import torch_geometric


import warnings
warnings.filterwarnings("ignore")

In [4]:
import os
import torch
import torch_geometric
import numpy as np
import pandas as pd
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader as graph_dataloader
from torch_geometric.utils import dense_to_sparse, remove_self_loops
import random

def seed_everything(seed = 0):
    r"""Sets the seed for generating random numbers in :pytorch:`PyTorch`,
    :obj:`numpy` and Python.

    Args:
        seed (int): The desired seed.
    """
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


def normalize(df_train, df_val, df_test):
    df_train_min = df_train.min()
    df_train_max = df_train.max()

    df_train_normalized = (df_train - df_train_min) / (df_train_max - df_train_min)
    df_val_normalized = (df_val - df_train_min) / (df_train_max - df_train_min)
    df_test_normalized = (df_test - df_train_min) / (df_train_max - df_train_min)


    return df_train_normalized, df_val_normalized, df_test_normalized


def split_dataframe(df, df_physics, train_ratio=0.8, val_ratio=0.1, mode='physics-enhanced'):
        train_ratio = int(0.8 * len(df.index))
        val_ratio = int(0.1 * len(df.index))
        df_train = df.iloc[:train_ratio]
        df_val = df.iloc[train_ratio:train_ratio+val_ratio]
        df_test = df.iloc[train_ratio+val_ratio:]
        df_train, df_val, df_test = normalize(df_train, df_val, df_test)
        df_X_train = df_train.iloc[:,37:]
        df_y_train = df_train.iloc[:,:37]
        df_X_val = df_val.iloc[:,37:]
        df_y_val = df_val.iloc[:,:37]
        df_X_test = df_test.iloc[:,37:]
        df_y_test = df_test.iloc[:,:37]

        if mode == 'physics-enhanced':
            df_physics_train = df_physics.iloc[:train_ratio]
            df_physics_val = df_physics.iloc[train_ratio:train_ratio+val_ratio]
            df_physics_test = df_physics.iloc[train_ratio+val_ratio:]
            df_physics_train, df_physics_val, df_physics_test = normalize(df_physics_train, df_physics_val, df_physics_test)

            df_X_train = pd.concat([df_X_train, df_physics_train], axis = 1)
            df_X_val = pd.concat([df_X_val, df_physics_val], axis = 1)
            df_X_test = pd.concat([df_X_test, df_physics_test], axis = 1)


        return np.array(df_X_train), np.array(df_y_train), np.array(df_X_val), np.array(df_y_val), np.array(df_X_test), np.array(df_y_test)


def gaussian_kernel_distance(feature1, feature2, sigma):
    # Calculate the Euclidean distance between two feature vectors
    distance = np.linalg.norm(feature1 - feature2)
    # Apply the Gaussian kernel function
    weight = np.exp(-distance**2 / (sigma**2))
    return weight


def construct_graph(dataset, threshold_factor=1.0):
    num_features = dataset.shape[1]
    adjacency_matrix = np.zeros((num_features, num_features))

    # Calculate pairwise distances
    pairwise_distances = np.zeros((num_features, num_features))
    for i in range(num_features):
        for j in range(i+1, num_features):
            pairwise_distances[i, j] = np.linalg.norm(dataset[:, i] - dataset[:, j])
            pairwise_distances[j, i] = pairwise_distances[i, j]

    # Calculate sigma as a multiple of the standard deviation of distances
    sigma = np.std(pairwise_distances) * threshold_factor

    # Construct the adjacency matrix with thresholding
    for i in range(num_features):
        for j in range(i+1, num_features):
            if pairwise_distances[i, j] <= threshold_factor:
                weight = gaussian_kernel_distance(dataset[:, i], dataset[:, j], sigma)
                adjacency_matrix[i, j] = weight
                adjacency_matrix[j, i] = weight

    return adjacency_matrix + np.identity(adjacency_matrix.shape[0])


def construct_pyg_data(df_X, df_y, device, window_size = 8):
    PyG_Data = []

    for i in range(df_X.T.shape[1] - window_size):
        start_idx = i
        end_idx = start_idx + window_size

        # Construct adjacency matrix and edge index
        adj = torch.from_numpy(construct_graph(df_X[start_idx:end_idx, :]).astype(float))
        edge_index = (adj > 0).nonzero().t()
        row, col = edge_index
        edge_weight = adj[row, col]

        # Convert NumPy arrays to PyTorch tensors
        x = torch.tensor(df_X.T[:, start_idx:end_idx], dtype=torch.float32)
        y = torch.tensor(df_y.T[:, start_idx:end_idx], dtype=torch.float32)
        edge_index = torch.tensor(edge_index, dtype=torch.long)
        edge_weight = torch.tensor(edge_weight, dtype=torch.float32)

        # Create PyG Data object and append to list
        data = Data(x=x, edge_index=edge_index, edge_attr=edge_weight, y=y).to(device)
        PyG_Data.append(data)

    return PyG_Data

from pathlib import Path
import argparse
import time

import torch
import random


from utils import *
%load /content/model.py
%load /content/train_test.py

import warnings
warnings.filterwarnings("ignore")

def get_arguments():
    parser = argparse.ArgumentParser(description="Physics-Enhanced GNN for Soft Sensing",
                                     add_help=False)

    # Data
    parser.add_argument("--data-dir", type=str, default="data/",
                        help='Path to the data')

    # Checkpoints
    parser.add_argument("--exp-dir", type=Path, default="exp/",
                        help='Path to the experiment folder, where all logs/checkpoints will be stored')

    # Optim
    parser.add_argument("--seed", type=int, default=42,
                        help='Seed for experiments')
    parser.add_argument("--epochs", type=int, default=25000,
                        help='Number of epochs')
    parser.add_argument("--batch-size", type=int, default=64,
                        help='Batch size')
    parser.add_argument("--base-lr", type=float, default=3e-4,
                        help='Base Learning rate')
    parser.add_argument("--window-size", type=int, default=8,
                        help='Window Size')
    parser.add_argument("--patience", type=int, default=200,
                        help='patience for early stopping')

    # Running
    parser.add_argument("--num-workers", type=int, default=1)
    parser.add_argument('--device', default='cuda:1',
                        help='device to use for training / testing')

    return parser


def seed_everything(seed = 0):
    r"""Sets the seed for generating random numbers in :pytorch:`PyTorch`,
    :obj:`numpy` and Python.

    Args:
        seed (int): The desired seed.
    """
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


def mains(args):
    device = args.device if torch.cuda.is_available() else 'cpu'

    df = pd.read_csv('/content/df_sensors (1).csv')
    df_physics = pd.read_csv('/content/df_physics (1).csv')

    df_X_train, df_y_train, df_X_val, df_y_val, df_X_test, df_y_test = split_dataframe(df, df_physics, train_ratio=0.8, val_ratio=0.1, mode='physics-enhanced')


    #Create Dataset
    PyG_Data_Train = construct_pyg_data(df_X_train, df_y_train, device)
    PyG_Data_Val = construct_pyg_data(df_X_val, df_y_val, device)
    PyG_Data_Test = construct_pyg_data(df_X_test, df_y_test, device)

    #Create Dataloader
    Train_DATA = graph_dataloader(PyG_Data_Train, batch_size = 64, shuffle = False, drop_last = True)
    Validation_DATA = graph_dataloader(PyG_Data_Val, batch_size = 64, shuffle = False, drop_last = True)
    Test_DATA = graph_dataloader(PyG_Data_Test, shuffle = False)

    #define Model
    model = GNNModel().to(device)

    trained_model = train_gnn_model(model, Train_DATA, Validation_DATA, device = device, window_size = args.window_size, patience = args.patience, EPOCHS = args.epochs, lr = args.base_lr)
    preds_list, targets_list, mse = test_gnn_model(trained_model, Test_DATA, window_size = args.window_size, device = device)


if __name__ == "__main__":
    parser = argparse.ArgumentParser(parents=[get_arguments()])
    args = parser.parse_args([])

    seed_everything(args.seed)
    mains(args)
    from torch import nn
from torch.nn import Parameter
import torch.nn.functional as F
import torch
from torch_geometric.nn import ChebConv, GATv2Conv, TransformerConv

class GNNModel(nn.Module):
    def __init__(self, input_dim = 8, hidden_dim = 16, out_dim = 8, num_heads = 5):
        super().__init__()
        self.GCN1 = GATv2Conv(input_dim, hidden_dim, heads = num_heads, add_self_loops = False)
        self.GCN2 = GATv2Conv(num_heads*hidden_dim, hidden_dim, heads = num_heads, add_self_loops = False)
        self.linear = nn.Linear(2* num_heads*hidden_dim, out_dim)
        self.linear1 = nn.Linear(38, 128)
        self.linear2 = nn.Linear(128 , 37)

    def forward(self, data):
      x, edge_index, edge_weight = data.x.float(), data.edge_index, data.edge_weight
      length = x.shape[1]
      x = F.selu(self.GCN1(x, edge_index, edge_weight))
      x_prev1 = x
      x = F.selu(self.GCN2(x, edge_index, edge_weight))
      x = torch.cat([x_prev1, x], dim = 1)
      x = F.selu(self.linear(x))
      x = F.selu(self.linear1(x.view(-1, 38, length).permute(0 , 2, 1)))
      x = self.linear2(x)
      return x
      import numpy as np
import torch
import torch.nn.functional as F
from torch.optim import NAdam

def train_gnn_model(model, Train_DATA, Validation_DATA, window_size, device, EPOCHS, lr, patience = 200):

    # Initialize model and optimizer
    optimizer = NAdam(model.parameters(), lr=lr)

    # Training parameters
    best_loss = float('inf')
    best_model = None
    counter = 0

    for epoch in range(EPOCHS):
        model.train()
        train_losses = []

        # Training loop
        for idx, data in enumerate(Train_DATA):
            optimizer.zero_grad()
            out = model(data)
            loss = F.mse_loss(out.squeeze(), data.y.float().view(-1, 37, window_size).permute(0, 2, 1).squeeze())
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        train_losses = np.array(train_losses)
        recon_epoch_loss = np.sqrt(train_losses.mean())
        total_epoch_loss = recon_epoch_loss

        # Print progress at every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f"Epoch: {epoch+1}")
            print('Training Loss:', recon_epoch_loss)

        val_losses = []

        # Validation loop
        val_loss = 0
        with torch.no_grad():
            for idx, data in enumerate(Validation_DATA):
                out = model(data)
                val_loss = F.mse_loss(out.squeeze(), data.y.float().view(-1, 37, window_size).permute(0, 2, 1).squeeze())
                val_losses.append(val_loss.item())

        val_losses = np.array(val_losses)
        recon_epoch_loss_eval = np.sqrt(val_losses.mean())
        total_epoch_loss_eval = recon_epoch_loss_eval

        # Print progress at every 10 epochs
        if (epoch + 1) % 10 == 0:
            print('Validation Loss:', recon_epoch_loss_eval)

        # Early stopping
        if total_epoch_loss_eval < best_loss:
            best_loss = total_epoch_loss_eval
            best_model = model.state_dict()
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print('Early stopping')
                break

    # Load the best model
    model.load_state_dict(best_model)

    return model


def test_gnn_model(model, Test_DATA, window_size, device):
    model.to(device)
    model.eval()
    preds_list = []
    targets_list = []
    cost = 0

    with torch.inference_mode():
        for idx, data in enumerate(Test_DATA):
            data = data.to(device)
            y_pred = model(data).squeeze()
            preds_list.append(y_pred.cpu().numpy())
            targets_list.append(data.y.float().view(-1, 37, window_size).permute(0, 2, 1).squeeze().cpu().numpy())
            cost += torch.mean((y_pred - data.y.float().view(-1, 37, window_size).permute(0, 2, 1).squeeze())**2).item()

    cost = cost / (idx + 1)
    mse = np.sqrt(cost)
    print(f"MSE: {mse:.6f}")

    return preds_list, targets_list, mse


NameError: name 'GNNModel' is not defined

In [ ]:
def get_arguments():
    parser = argparse.ArgumentParser(description="Physics-Enhanced GNN for Soft Sensing",
                                     add_help=False)

    # Data
    parser.add_argument("--data-dir", type=str, default="data/",
                        help='Path to the data')

    # Checkpoints
    parser.add_argument("--exp-dir", type=Path, default="exp/",
                        help='Path to the experiment folder, where all logs/checkpoints will be stored')

    # Optim
    parser.add_argument("--seed", type=int, default=42,
                        help='Seed for experiments')
    parser.add_argument("--epochs", type=int, default=25000,
                        help='Number of epochs')
    parser.add_argument("--batch-size", type=int, default=64,
                        help='Batch size')
    parser.add_argument("--base-lr", type=float, default=3e-4,
                        help='Base Learning rate')
    parser.add_argument("--window-size", type=int, default=8,
                        help='Window Size')
    parser.add_argument("--patience", type=int, default=200,
                        help='patience for early stopping')

    # Running
    parser.add_argument("--num-workers", type=int, default=1)
    parser.add_argument('--device', default='cuda:1',
                        help='device to use for training / testing')

    return parser

In [ ]:
def seed_everything(seed = 0):
    r"""Sets the seed for generating random numbers in :pytorch:`PyTorch`,
    :obj:`numpy` and Python.

    Args:
        seed (int): The desired seed.
    """
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [ ]:
def mains(args):
    device = args.device if torch.cuda.is_available() else 'cpu'

    df = pd.read_csv('/content/df_sensors (1).csv')
    df_physics = pd.read_csv('Data/df_physics.csv')

    df_X_train, df_y_train, df_X_val, df_y_val, df_X_test, df_y_test = split_dataframe(df, df_physics, train_ratio=0.8, val_ratio=0.1, mode='physics-enhanced')


    #Create Dataset
    PyG_Data_Train = construct_pyg_data(df_X_train, df_y_train, device)
    PyG_Data_Val = construct_pyg_data(df_X_val, df_y_val, device)
    PyG_Data_Test = construct_pyg_data(df_X_test, df_y_test, device)

    #Create Dataloader
    Train_DATA = graph_dataloader(PyG_Data_Train, batch_size = 64, shuffle = False, drop_last = True)
    Validation_DATA = graph_dataloader(PyG_Data_Val, batch_size = 64, shuffle = False, drop_last = True)
    Test_DATA = graph_dataloader(PyG_Data_Test, shuffle = False)

    #define Model
    model = GNNModel().to(device)

    trained_model = train_gnn_model(model, Train_DATA, Validation_DATA, device = device, window_size = args.window_size, patience = args.patience, EPOCHS = args.epochs, lr = args.base_lr)
    preds_list, targets_list, mse = test_gnn_model(trained_model, Test_DATA, window_size = args.window_size, device = device)

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser(parents=[get_arguments()])
    args = parser.parse_args([])

    seed_everything(args.seed)
    mains(args)

In [6]:
import os
import random
import warnings
import argparse
from pathlib import Path
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch.optim import NAdam
import numpy as np
import pandas as pd
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader as graph_dataloader
from torch_geometric.nn import GATv2Conv
from torch_geometric.utils import dense_to_sparse, remove_self_loops

# Ignore warnings
warnings.filterwarnings("ignore")

# ---------------------- Utility Functions ----------------------

def seed_everything(seed=0):
    """Sets the seed for generating random numbers in various libraries."""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

def normalize(df_train, df_val, df_test):
    """Normalizes train, validation, and test datasets based on training data statistics."""
    df_train_min = df_train.min()
    df_train_max = df_train.max()

    df_train_normalized = (df_train - df_train_min) / (df_train_max - df_train_min)
    df_val_normalized = (df_val - df_train_min) / (df_train_max - df_train_min)
    df_test_normalized = (df_test - df_train_min) / (df_train_max - df_train_min)

    return df_train_normalized, df_val_normalized, df_test_normalized

def split_dataframe(df, df_physics, train_ratio=0.8, val_ratio=0.1, mode='physics-enhanced'):
    """Splits the dataframe into train, validation, and test sets."""
    train_size = int(train_ratio * len(df))
    val_size = int(val_ratio * len(df))

    df_train = df.iloc[:train_size]
    df_val = df.iloc[train_size:train_size + val_size]
    df_test = df.iloc[train_size + val_size:]

    df_train, df_val, df_test = normalize(df_train, df_val, df_test)

    df_X_train = df_train.iloc[:, 37:]
    df_y_train = df_train.iloc[:, :37]
    df_X_val = df_val.iloc[:, 37:]
    df_y_val = df_val.iloc[:, :37]
    df_X_test = df_test.iloc[:, 37:]
    df_y_test = df_test.iloc[:, :37]

    if mode == 'physics-enhanced':
        df_physics_train = df_physics.iloc[:train_size]
        df_physics_val = df_physics.iloc[train_size:train_size + val_size]
        df_physics_test = df_physics.iloc[train_size + val_size:]

        df_physics_train, df_physics_val, df_physics_test = normalize(
            df_physics_train, df_physics_val, df_physics_test)

        df_X_train = pd.concat([df_X_train, df_physics_train], axis=1)
        df_X_val = pd.concat([df_X_val, df_physics_val], axis=1)
        df_X_test = pd.concat([df_X_test, df_physics_test], axis=1)

    return np.array(df_X_train), np.array(df_y_train), np.array(df_X_val), np.array(df_y_val), np.array(df_X_test), np.array(df_y_test)

def gaussian_kernel_distance(feature1, feature2, sigma):
    """Computes the Gaussian kernel distance between two feature vectors."""
    distance = np.linalg.norm(feature1 - feature2)
    weight = np.exp(-distance**2 / (sigma**2))
    return weight

def construct_graph(dataset, threshold_factor=1.0):
    """Constructs an adjacency matrix for the dataset based on a Gaussian kernel."""
    num_features = dataset.shape[1]
    adjacency_matrix = np.zeros((num_features, num_features))

    pairwise_distances = np.zeros((num_features, num_features))
    for i in range(num_features):
        for j in range(i + 1, num_features):
            pairwise_distances[i, j] = np.linalg.norm(dataset[:, i] - dataset[:, j])
            pairwise_distances[j, i] = pairwise_distances[i, j]

    sigma = np.std(pairwise_distances) * threshold_factor

    for i in range(num_features):
        for j in range(i + 1, num_features):
            if pairwise_distances[i, j] <= threshold_factor:
                weight = gaussian_kernel_distance(dataset[:, i], dataset[:, j], sigma)
                adjacency_matrix[i, j] = weight
                adjacency_matrix[j, i] = weight

    return adjacency_matrix + np.identity(adjacency_matrix.shape[0])

def construct_pyg_data(df_X, df_y, device, window_size=8):
    """Constructs PyTorch Geometric Data objects for the dataset."""
    PyG_Data = []

    for i in range(df_X.T.shape[1] - window_size):
        start_idx = i
        end_idx = start_idx + window_size

        adj = torch.from_numpy(construct_graph(df_X[start_idx:end_idx, :]).astype(float))
        edge_index = (adj > 0).nonzero().t()
        row, col = edge_index
        edge_weight = adj[row, col]

        x = torch.tensor(df_X.T[:, start_idx:end_idx], dtype=torch.float32)
        y = torch.tensor(df_y.T[:, start_idx:end_idx], dtype=torch.float32)
        edge_index = torch.tensor(edge_index, dtype=torch.long)
        edge_weight = torch.tensor(edge_weight, dtype=torch.float32)

        data = Data(x=x, edge_index=edge_index, edge_attr=edge_weight, y=y).to(device)
        PyG_Data.append(data)

    return PyG_Data

# ---------------------- Model Definition ----------------------

class GNNModel(nn.Module):
    def __init__(self, input_dim=8, hidden_dim=16, out_dim=8, num_heads=5):
        super().__init__()
        self.GCN1 = GATv2Conv(input_dim, hidden_dim, heads=num_heads, add_self_loops=False)
        self.GCN2 = GATv2Conv(num_heads * hidden_dim, hidden_dim, heads=num_heads, add_self_loops=False)
        self.linear = nn.Linear(2 * num_heads * hidden_dim, out_dim)
        self.linear1 = nn.Linear(38, 128)
        self.linear2 = nn.Linear(128, 37)

    def forward(self, data):
        x, edge_index, edge_weight = data.x.float(), data.edge_index, data.edge_weight
        length = x.shape[1]
        x = F.selu(self.GCN1(x, edge_index, edge_weight))
        x_prev1 = x
        x = F.selu(self.GCN2(x, edge_index, edge_weight))
        x = torch.cat([x_prev1, x], dim=1)
        x = F.selu(self.linear(x))
        x = F.selu(self.linear1(x.view(-1, 38, length).permute(0, 2, 1)))
        x = self.linear2(x)
        return x

# ---------------------- Training and Evaluation ----------------------

def train_gnn_model(model, Train_DATA, Validation_DATA, window_size, device, EPOCHS, lr, patience=200):
    """Trains the GNN model."""
    optimizer = NAdam(model.parameters(), lr=lr)
    best_loss = float('inf')
    best_model = None
    counter = 0

    for epoch in range(EPOCHS):
        model.train()
        train_losses = []

        for idx, data in enumerate(Train_DATA):
            optimizer.zero_grad()
            out = model(data)
            loss = F.mse_loss(out.squeeze(), data.y.float().view(-1, 37, window_size).permute(0, 2, 1))
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        avg_train_loss = np.mean(train_losses)

        # Validation Loop
        model.eval()
        val_losses = []

        with torch.no_grad():
            for data in Validation_DATA:
                out = model(data)
                loss = F.mse_loss(out.squeeze(), data.y.float().view(-1, 37, window_size).permute(0, 2, 1))
                val_losses.append(loss.item())

        avg_val_loss = np.mean(val_losses)

        # Early Stopping
        if avg_val_loss < best_loss:
            best_loss
